# Assignment 4 - Part 2

In [5]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression  
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
import sklearn
import random
sns.set()



In [6]:
df = pd.read_csv('airquality.csv',usecols = ['Ozone','Solar.R','Wind','Temp'])
df.head()

,Ozone,Solar.R,Wind,Temp
0,41.0,190.0,7.4,67
1,36.0,118.0,8.0,72
2,12.0,149.0,12.6,74
3,18.0,313.0,11.5,62
4,NaN,NaN,14.3,56


## Assignment 4 - Part 2 - excercise1-missing Data Report

In [7]:
df_airquality = df.copy()
print(df_airquality.isna().any())
print(df_airquality.isna().sum())   # Reporting the number of Null values

Ozone       True
Solar.R     True
Wind       False
Temp       False
dtype: bool
Ozone      37
Solar.R     7
Wind        0
Temp        0
dtype: int64


## Assignment 4 - Part 2 - excercise1- dropping- missing Data 

In [8]:
df_airquality = df_airquality.dropna()

#df_airquality.isna().any()
df_airquality.head()


,Ozone,Solar.R,Wind,Temp
0,41.0,190.0,7.4,67
1,36.0,118.0,8.0,72
2,12.0,149.0,12.6,74
3,18.0,313.0,11.5,62
6,23.0,299.0,8.6,65


## Assignment 4 - Part 2 - excercise2- 

## which observation are in validation set
### New column for interaction formula

In [9]:
df_airquality['interaction_formula'] = df_airquality['Solar.R'] + \
        df_airquality['Wind'] + df_airquality['Temp'] + df_airquality['Solar.R']**2 + df_airquality['Wind']**2 +\
        df_airquality['Temp']**2 + df_airquality['Solar.R']* df_airquality['Wind'] + df_airquality['Temp']* df_airquality['Solar.R']+\
        df_airquality['Temp']* df_airquality['Wind']

df_airquality.head()

,Ozone,Solar.R,Wind,Temp,interaction_formula
0,41.0,190.0,7.4,67,55539.96
1,36.0,118.0,8.0,72,29386.00
2,12.0,149.0,12.6,74,41907.16
3,18.0,313.0,11.5,62,126050.25
6,23.0,299.0,8.6,65,116637.96


In [10]:
x_train,x_valid,y_train,y_valid = train_test_split(df_airquality[['Solar.R','Wind','Temp','interaction_formula']],df_airquality['Ozone'],
                                                  train_size = 0.75,random_state = 4099183)
print(f"observations in validation set as features are\n\n {x_valid.head()}\n") #{x_valid.shape}\n\n")
print('x_valid shape is :', x_valid.shape)
print(f"\n\nobservations in validation set as respone is\n {y_valid.head()}\n")# {y_valid.shape} ")
print('y_valid shape is :', y_valid.shape)

observations in validation set as features are

      Solar.R  Wind  Temp  interaction_formula
113     36.0  14.3    72             10943.19
72     264.0  14.3    73             99671.89
140     27.0  10.3    76              9837.29
67     276.0   5.1    88            110459.51
40     323.0  11.5    87            145267.75

x_valid shape is : (28, 4)


observations in validation set as respone is
 113     9.0
72     10.0
140    13.0
67     77.0
40     39.0
Name: Ozone, dtype: float64

y_valid shape is : (28,)


In [11]:
# Validation set
print(y_valid.head())

113     9.0
72     10.0
140    13.0
67     77.0
40     39.0
Name: Ozone, dtype: float64


## Assignment 4 - Part 2- exercise3 - fit Models and Report MSPE


In [12]:
reg_Solar = LinearRegression().fit(x_train[['Solar.R']],y_train)  
reg_Wind = LinearRegression().fit(x_train[['Wind']],y_train)
reg_Temp = LinearRegression().fit(x_train[['Temp']],y_train)
reg_interaction_formula = LinearRegression().fit(x_train[['interaction_formula']],y_train)
reg_Solar_Wind_Temp = LinearRegression().fit(x_train[['Solar.R','Wind','Temp']] ,y_train )

In [13]:
Solar_predict= reg_Solar.predict(x_valid[['Solar.R']])

Wind_predict= reg_Wind.predict(x_valid[['Wind']])

Temp_predict= reg_Temp.predict(x_valid[['Temp']])

interaction_formula_predict = reg_interaction_formula.predict(x_valid[['interaction_formula']])

Solar_Wind_Temp_predict = reg_Solar_Wind_Temp.predict(x_valid[['Solar.R','Wind','Temp']])

### Report MSPE

In [14]:
Solar_predict_MSPE = round(sklearn.metrics.mean_squared_error(y_valid,Solar_predict),2)

wind_predict_MSPE =round( sklearn.metrics.mean_squared_error(y_valid,Wind_predict),2)

Temp_predict_MSPE = round(sklearn.metrics.mean_squared_error(y_valid,Temp_predict),2)

Solar_Wind_Temp_MSPE  = round(sklearn.metrics.mean_squared_error(y_valid ,Solar_Wind_Temp_predict),2)

interaction_formula_MSPE = round(sklearn.metrics.mean_squared_error(y_valid,interaction_formula_predict),2)

print(f'Solar_predict_MSPE is : {Solar_predict_MSPE}')
print(f'wind_predict_MSPE is :{wind_predict_MSPE}')
print(f"Temp_predict_MSPE is : {Temp_predict_MSPE}")
print(f"Solar_Wind_Temp_MSPE is :{Solar_Wind_Temp_MSPE} ")
print(f"interaction_formula_MSPE is :{interaction_formula_MSPE}" )


Solar_predict_MSPE is : 1163.08
wind_predict_MSPE is :890.21
Temp_predict_MSPE is : 620.32
Solar_Wind_Temp_MSPE is :473.13 
interaction_formula_MSPE is :1211.15


## **Answer 3-a**

According to calculated MPSE, The winner is 3 variables regression (model 4) Solar_Wind_Temp with MPSE = 473

# Assignment 4 - Part 2 - excrsize 4 - Cross Validation
**The reason of using *'neg_mean_squared_error'* in codes:**

**All scorer objects follow the convention that higher return values are better than lower return values. Thus metrics which measure the distance between the model and the data, like *'metrics.mean_squared_error'*, are available as *'neg_mean_squared_error'* which return the negated value of the metric.**

In [15]:
rcv = RepeatedKFold(n_splits= 10, n_repeats= 1, random_state= 4099183)

score_cv_sloar = cross_val_score(reg_Solar, x_valid[['Solar.R']], y_valid, scoring= 'neg_mean_squared_error', cv= rcv)
score_cv_wind = cross_val_score(reg_Wind, x_valid[['Wind']], y_valid, scoring= 'neg_mean_squared_error', cv= rcv)
score_cv_temp = cross_val_score(reg_Temp, x_valid[['Temp']], y_valid, scoring= 'neg_mean_squared_error', cv= rcv)
score_cv_inter = cross_val_score(reg_interaction_formula, x_valid[['interaction_formula']], y_valid, scoring= 'neg_mean_squared_error', cv= rcv)
score_cv_Solar_Wind_Temp = cross_val_score(reg_Solar_Wind_Temp, x_valid[['Solar.R','Wind','Temp']], y_valid, scoring= 'neg_mean_squared_error', cv= rcv)

print('Mean Squared Predict Error for Solar is : %.2f' %-score_cv_sloar.mean())
print('upper bound of confidence interval for solars is:', round( -score_cv_sloar.mean()+ 1.96 * score_cv_sloar.std(),2))
print('lower bound of confidence interval for solars is:', round( -score_cv_sloar.mean()- 1.96 * score_cv_sloar.std(),2))
print()

print('Mean Squared Predict Error for Wind is : %.2f' %-score_cv_wind.mean())
print('upper bound of confidence interval for Wind is:', round( -score_cv_wind.mean()+ 1.96 * score_cv_sloar.std(),2))
print('lower bound of confidence interval for Wind is:', round( -score_cv_wind.mean()- 1.96 * score_cv_sloar.std(),2))
print()

print('Mean Squared Predict Error for Temp is : %.2f' %-score_cv_temp.mean())
print('upper bound of confidence interval for temp is:', round( -score_cv_temp.mean()+ 1.96 * score_cv_temp.std(),2))
print('lower bound of confidence interval for temp is:', round( -score_cv_temp.mean()- 1.96 * score_cv_temp.std(),2))
print()

print('Mean Squared Predict Error for Interaction Formula is : %.2f' %-score_cv_inter.mean())
print('upper bound of confidence interval for score_cv_inter is:', round( -score_cv_inter.mean()+ 1.96 * score_cv_inter.std(),2))
print('lower bound of confidence interval for score_cv_inter is:', round( -score_cv_inter.mean()- 1.96 * score_cv_inter.std(),2))
print()

print('Mean Squared Predict Error for Solar, Wind and Temp is : %.2f' %-score_cv_Solar_Wind_Temp.mean())
print('upper bound of confidence interval for score_cv_Solar_Wind_Temp is:', round( -score_cv_Solar_Wind_Temp.mean()+ 1.96 * score_cv_Solar_Wind_Temp.std(),2))
print('lower bound of confidence interval for score_cv_Solar_Wind_Temp is:', round( -score_cv_Solar_Wind_Temp.mean()- 1.96 * score_cv_Solar_Wind_Temp.std(),2))



Mean Squared Predict Error for Solar is : 1250.05
upper bound of confidence interval for solars is: 2792.71
lower bound of confidence interval for solars is: -292.62

Mean Squared Predict Error for Wind is : 1013.99
upper bound of confidence interval for Wind is: 2556.65
lower bound of confidence interval for Wind is: -528.68

Mean Squared Predict Error for Temp is : 642.02
upper bound of confidence interval for temp is: 2094.14
lower bound of confidence interval for temp is: -810.1

Mean Squared Predict Error for Interaction Formula is : 1326.81
upper bound of confidence interval for score_cv_inter is: 2924.68
lower bound of confidence interval for score_cv_inter is: -271.06

Mean Squared Predict Error for Solar, Wind and Temp is : 593.04
upper bound of confidence interval for score_cv_Solar_Wind_Temp is: 1631.3
lower bound of confidence interval for score_cv_Solar_Wind_Temp is: -445.23


## Answer 4-a

According to the Cross Validation output with MSE scoring, the best model is Solar, Wind and Temp variables (3 Variables) regression which is clearly good (mse = 464) and

Lonely Solar (mse = 920) and Interaction(mse = 927) seems to be clearly bad Models.



## Assignment 4 - Part 2 - excrsize 4 - Cross Validation for 10 folds and 20 Repeats

In [16]:
rcv = RepeatedKFold(n_splits= 10, n_repeats= 20, random_state= 4099183)

score_cv_sloar = cross_val_score(reg_Solar, , y_valid, scoring= 'neg_mean_squared_error', cv= rcv)
score_cv_wind = cross_val_score(reg_Wind, x_valid[['Wind']], y_valid, scoring= 'neg_mean_squared_error', cv= rcv)
score_cv_temp = cross_val_score(reg_Temp, x_valid[['Temp']], y_valid, scoring= 'neg_mean_squared_error', cv= rcv)
score_cv_inter = cross_val_score(reg_interaction_formula, x_valid[['interaction_formula']], y_valid, scoring= 'neg_mean_squared_error', cv= rcv)
score_cv_Solar_Wind_Temp = cross_val_score(reg_Solar_Wind_Temp, x_valid[['Solar.R','Wind','Temp']], y_valid, scoring= 'neg_mean_squared_error', cv= rcv)

print('Mean Squared Predict Error for Solar is : %.2f' %-score_cv_sloar.mean())
print('Mean Squared Predict Error for Wind is : %.2f' %-score_cv_wind.mean())
print('Mean Squared Predict Error for Temp is : %.2f' %-score_cv_temp.mean())
print('Mean Squared Predict Error for Interaction Formula is : %.2f' %-score_cv_inter.mean())
print('Mean Squared Predict Error for Solar, Wind and Temp is : %.2f' %-score_cv_Solar_Wind_Temp.mean())


SyntaxError: invalid syntax (<ipython-input-16-c6a818cfbede>, line 3)

In [ ]:
x_valid[['Solar.R','Wind','Temp']]

## Box Plots

In [ ]:
fig, ax = plt.subplots(2,3, figsize=(12,8), sharey = True)

ax[0,0].boxplot(-score_cv_sloar, sym='', meanline= True, showmeans= True)
ax[0,0].set_title('1- Solar')
ax[0,0].annotate('MSPE = 1292.68', xy = (0.5, 2500))

ax[0,1].boxplot(-score_cv_wind, sym='', meanline= True, showmeans= True)
ax[0,1].set_title('2- Wind')
ax[0,1].annotate('MSPE = 1045.91', xy = (0.5, 2500))

ax[0,2].boxplot(-score_cv_temp, sym='',meanline= True, showmeans= True)
ax[0,2].set_title('3- Temp')
ax[0,2].annotate('MSPE = 624.69', xy = (0.5, 2500))

ax[1,0].boxplot(-score_cv_Solar_Wind_Temp, sym='',meanline= True, showmeans= True)
ax[1,0].set_title('4-variables')
ax[1,0].annotate('MSPE = 584.85', xy = (0.5, 2500))

ax[1,1].boxplot(-score_cv_inter, sym='',meanline= True, showmeans= True)
ax[1,1].set_title('5- Interaction')
ax[1,1].annotate('MSPE = 1362.91', xy = (0.5, 2500))

plt.show()

In [ ]:
plt.boxplot(-score_cv_sloar, sym='', meanline= True, showmeans= True)
plt.show()

In [ ]:
plt.boxplot(-score_cv_sloar, sym='')
plt.show()

### Zoom in to see two plots better

In [ ]:
fig1, ax1 = plt.subplots(1,2, figsize=(12,8), sharey = True)


ax1[0].boxplot(-score_cv_temp, sym='')
ax1[0].set_title('3- Temp')
ax1[0].annotate('MSPE = 599.57', xy = (0.5, 2500))

ax1[1].boxplot(-score_cv_Solar_Wind_Temp, sym='')
ax1[1].set_title('4-variables')
ax1[1].annotate('MSPE = 589.08', xy = (0.5, 2500))



plt.show()

## Answer 5-a



**boxplot(1-solar) : I see in the chart that the 95% of confidence interval is between almost 0 to 3000 and median is about 1000. The box shows the lower quartile about 500 and upper quartile about 2000. So the variation of MSPE is too much and it isn't good model for prediction**
    
**boxplot(2-wind):I see in the chart that the 95% of confidence interval is between almost 250 to 2000 and median is about 1000. The box shows the lower quartile about 500 and upper quartile about 1250. So the variation of MSPE is too much and it isn't good model for prediction**
    
**boxplot(3-Temp):I see in the chart that the 95% of confidence interval is between almost 0 to 900 and median is under 500. The box shows the lower quartile about 300 and upper quartile about 700. So the variation of MSPE is much less than the previous models and it seems to be a better model.**
    
**boxplot(4- 3variables):I see in the chart that the 95% of confidence interval is between almost 100 to 1500 and median is under 500. The box shows the lower quartile about 300 and upper quartile about 800. So the variation of MSPE is much less than the previous models(1 and 2) and it seems to be a better model.**
    
**boxplot(5-Interaction):I see in the chart that the 95% of confidence interval is between almost 250 to 3000 and median is about 1200. The box shows the lower quartile about 500 and upper quartile about 2000. So the variation of MSPE is too much and it isn't good model for prediction**

## 5-b Box Plots of RMSPE

In [ ]:
RMSPE_Solar = -score_cv_sloar/(-score_cv_sloar).min()
RMSPE_Wind = -score_cv_wind/(-score_cv_wind).min()
RMSPE_Temp = -score_cv_temp/(-score_cv_temp).min()
RMSPE_Solar_Wind_Temp = -score_cv_Solar_Wind_Temp/(-score_cv_Solar_Wind_Temp).min()
RMSPE_inter = -score_cv_inter/(-score_cv_inter).min()


In [ ]:
fig2, ax2 = plt.subplots(2,3, sharey= True, figsize=(12,8))

ax2[0,0].boxplot(RMSPE_Solar, sym= '')

ax2[0,1].boxplot(RMSPE_Wind, sym= '')

ax2[0,2].boxplot(RMSPE_Temp, sym= '')

ax2[1,0].boxplot(RMSPE_Solar_Wind_Temp, sym= '')

ax2[1,1].boxplot(RMSPE_inter, sym= '')

ax2[0,0].set_title('1- Solar')
ax2[0,1].set_title('2- Wind')
ax2[0,2].set_title('3- Temp')
ax2[1,0].set_title('4-variables')
ax2[1,1].set_title('5- Interaction')
plt.show()

In [ ]:
fig3, ax3 = plt.subplots(1,3, sharey= True, figsize=(12,8))


ax3[0].boxplot(RMSPE_Wind, sym= '')

ax3[1].boxplot(RMSPE_Temp, sym= '')

ax3[2].boxplot(RMSPE_Solar_Wind_Temp, sym= '')

ax3[0].set_title('2- Wind')
ax3[1].set_title('3- Temp')
ax3[2].set_title('4-variables')

plt.show()

## Answer 5-b

I see in the charts that the relative MSPE is between 1 and maximum almost 45 in Solar regression.

As I see in the charts, the Temprature and 3 variables are better models as it was seen in MSPE too.

One other model that looks good in RMSPE charts belong to Wind regression.

## Answer 6

**If I have to select one model, from different model analysis, the 3 variables (Solar, Wind, and temp) is recommanded.**